In [6]:
import sys
print(sys.version)

3.5.2 (default, Aug 18 2017, 17:48:00) 
[GCC 5.4.0 20160609]


In [3]:
#! pip3 install pandas
! sudo pip3 install scipy

The directory '/home/FDSM_lhn/.cache/pip/http' or its parent directory is not owned by the current user and the cache has been disabled. Please check the permissions and owner of that directory. If executing pip with sudo, you may want sudo's -H flag.
The directory '/home/FDSM_lhn/.cache/pip' or its parent directory is not owned by the current user and caching wheels has been disabled. check the permissions and owner of that directory. If executing pip with sudo, you may want sudo's -H flag.
    100% |████████████████████████████████| 47.9MB 33kB/s  eta 0:00:01


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
# train = pd.read_csv('datasets/train.csv')

# # train.head
# train.iloc[:10,:10]
# N, _ = train.shape

# for i in range(3):
#     file = "datasets/train_batch_{}".format(i)
#     with open(file,'wb') as f:
#         pickle.dump(train.iloc[int(2e5)*i:min(N+1,int(2e5) * (i+1)),:] ,file = f)
    
# test = pd.read_csv('datasets/test.csv')
# test.shape

# N, _ = test.shape

# for i in range(5):
#     file = "datasets/test_batch_{}".format(i)
#     with open(file,'wb') as f:
#         pickle.dump(test.iloc[int(2e5)*i:min(N+1,int(2e5) * (i+1)),:] ,file = f)

In [5]:
import data_util
import data_preprocess
import datetime

tic= datetime.datetime.now()
train_data = data_util.load_train_data()
test_data= data_util.load_test_data()

naive_pre = data_preprocess.naive_preprocess()
train_data = naive_pre.dtype_change(train_data)
test_data = naive_pre.dtype_change(test_data)
#train_prop is specified
X_train, X_val, y_train, y_val = data_util.split_train(train_data,prop=0.75)

X_train = naive_pre.scale(X_train)
X_val = naive_pre.scale(X_val,test=True)


/usr/local/lib/python3.5/dist-packages/sklearn/model_selection/_split.py:2010: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [6]:
print(X_train.shape ,X_val.shape)
# dev_train.dtypes
# dev_train.head
# train_data.isnull().any()
import 

(446409, 235) (148803, 235)


In [29]:
from sklearn.neural_network import MLPClassifier

tic= datetime.datetime.now()
hidden_layer_size = [(300,200,100)]
learning_rate = [0.001]
reg = []
momentum =[0.9]
rso= np.random.RandomState(66)

#for now I don't mess up with inits
mlp = MLPClassifier(hidden_layer_sizes= hidden_layer_size[0],random_state=rso,alpha= 0.001)
mlp.fit(X_train,y_train)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100, 100, 100), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5,
       random_state=<mtrand.RandomState object at 0x7fb955bcc480>,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [30]:
#check the performance
from sklearn.metrics import classification_report,confusion_matrix,roc_auc_score
test_predictions = mlp.predict_proba(X_val)
auc_score = roc_auc_score(y_val,test_predictions[:,1])

print('The val score it have: {} '.format(auc_score))
print("The time it cost: {}".format(toc-tic))

test_predictions = mlp.predict_proba(X_train)
auc_score = roc_auc_score(y_train,test_predictions[:,1])
toc = datetime.datetime.now()
print('The test score it have: {} '.format(auc_score))
print("The time it cost: {}".format(toc-tic))

#classification_report(y_train,test_predictions[:,1])

The val score it have: 0.5953008753764144 
The time it cost: -1 day, 23:57:44.336906
The test score it have: 0.7112570696063083 
The time it cost: 0:29:13.188801


In [20]:
roc_auc_score(y_val,test_predictions[:,1])
# sum(test_predictions)
# sum(dev_y)
# test_predictions
# sum(test_predictions[:,0])

0.62780916443936541

In [15]:
hidden_layer_size[0]

(50, 30)

# Try Different Package for NN

In [21]:
import torch 
import torch.utils.data as Data
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F

input_size = 235
hidden_size = [300,250,200]
num_classes = 2
num_epochs = 5
batch_size = 5000
learning_rate = 0.0005
weight_decay = 1e-5

# X_dev_tensor = torch.from_numpy(X_dev.values)
# y_dev_tensor = torch.from_numpy(y_dev.values)

# X_dev_tensor= X_dev_tensor.float()
# y_dev_tensor =y_dev_tensor.type(torch.LongTensor)
# #print(X_train_tensor)
# X_dev_Variable = Variable(X_dev_tensor)
# y_dev_Variable = Variable(y_dev_tensor)

X_train_tensor = torch.from_numpy(X_train.values)
y_train_tensor = torch.from_numpy(y_train.values)

X_train_tensor= X_train_tensor.float()
y_train_tensor =y_train_tensor.type(torch.LongTensor)
#print(X_train_tensor)
X_train_Variable = Variable(X_train_tensor)
y_train_Variable = Variable(y_train_tensor)


train_set = Data.TensorDataset(data_tensor=X_train_tensor, target_tensor=y_train_tensor )



train_loader = Data.DataLoader(
    dataset=train_set,      
    batch_size=batch_size,      # mini batch size
    shuffle=True,               # random shuffle for training
    num_workers=2)              # subprocesses for loading data


# train_loader = Data.DataLoader(dataset=train_dataset, 
#                                            batch_size=batch_size, 
#                                            shuffle=True)

# test_loader = Data.DataLoader(dataset=test_dataset, 
#                                           batch_size=batch_size, 
#                                           shuffle=False)
    
class My_Net(nn.Module):
    def __init__(self,input_size, hidden_size, num_classes):
        super(My_Net,self).__init__()
        self.fc1 = nn.Linear(input_size,hidden_size[0])
        self.fc2 = nn.Linear(hidden_size[0],hidden_size[1])
        self.fc3 = nn.Linear(hidden_size[1],hidden_size[2])
        self.fc4 = nn.Linear(hidden_size[2], num_classes)
        self.relu = nn.ReLU()
        
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.relu(out)
        out = self.fc3(out)
        out = self.relu(out)
        out = self.fc4(out)
        return out
        
net = My_Net(input_size, hidden_size,num_classes)
criterion = nn.CrossEntropyLoss()  
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate,weight_decay=weight_decay)

for epoch in range(num_epochs):
    for i,  (batch_x, batch_y) in enumerate(train_loader):
#         print(batch_x)
        x,  y = Variable(batch_x), Variable(batch_y)
        out = net(x)
        loss = criterion(out, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if i % 10 ==0:
            print('epoch {}: iteration {}, the loss is {}'.format(epoch,i ,loss))

epoch 0: iteration 0, the loss is Variable containing:
 0.6053
[torch.FloatTensor of size 1]

epoch 0: iteration 10, the loss is Variable containing:
 0.1907
[torch.FloatTensor of size 1]

epoch 0: iteration 20, the loss is Variable containing:
 0.1484
[torch.FloatTensor of size 1]

epoch 0: iteration 30, the loss is Variable containing:
 0.1588
[torch.FloatTensor of size 1]

epoch 0: iteration 40, the loss is Variable containing:
 0.1638
[torch.FloatTensor of size 1]

epoch 0: iteration 50, the loss is Variable containing:
 0.1498
[torch.FloatTensor of size 1]

epoch 0: iteration 60, the loss is Variable containing:
 0.1580
[torch.FloatTensor of size 1]

epoch 0: iteration 70, the loss is Variable containing:
 0.1453
[torch.FloatTensor of size 1]

epoch 0: iteration 80, the loss is Variable containing:
 0.1545
[torch.FloatTensor of size 1]

epoch 0: iteration 90, the loss is Variable containing:
 0.1562
[torch.FloatTensor of size 1]

epoch 0: iteration 100, the loss is Variable contai

In [15]:
from sklearn.metrics import roc_auc_score            
#try to test it's accuracy
out=  net(X_train_Variable)
out = F.softmax(out)
score_1 = roc_auc_score(y_train_Variable.data.numpy(), out.data.numpy()[:,1])

X_val_tensor = torch.from_numpy(X_val.values)
y_val_tensor = torch.from_numpy(y_val.values)

X_val_tensor= X_val_tensor.float()
y_val_tensor =y_val_tensor.type(torch.LongTensor)
#print(X_train_tensor)
X_val_Variable = Variable(X_val_tensor)
y_val_Variable = Variable(y_val_tensor)
out2=  net(X_val_Variable)
out2 = F.softmax(out2)
score_2 = roc_auc_score(y_val_Variable.data.numpy(), out2.data.numpy()[:,1])

print(out.data.numpy()[:,1].shape)
print(out2.data.numpy()[:,1].shape)
print('test set auc is {}; Val set auc is {}'.format(score_1, score_2))


#So here I've tested that with Variable type, I can run this function
# for t in range(100):
#     out = net(X_train_tensor)                 # input x and predict based on x
#     loss = criterion(out, y_train_tensor)     # must be (1. nn output, 2. target), the target label is NOT one-hotted

#     optimizer.zero_grad()   # clear gradients for next train
#     loss.backward()         # backpropagation, compute gradients
#     optimizer.step()

epoch 0: iteration 0, the loss is Variable containing:
 0.7395
[torch.FloatTensor of size 1]

epoch 0: iteration 10, the loss is Variable containing:
 0.2306
[torch.FloatTensor of size 1]

epoch 0: iteration 20, the loss is Variable containing:
 0.1929
[torch.FloatTensor of size 1]

epoch 0: iteration 30, the loss is Variable containing:
 0.1489
[torch.FloatTensor of size 1]

epoch 0: iteration 40, the loss is Variable containing:
 0.1502
[torch.FloatTensor of size 1]

epoch 0: iteration 50, the loss is Variable containing:
 0.1642
[torch.FloatTensor of size 1]

epoch 0: iteration 60, the loss is Variable containing:
 0.1539
[torch.FloatTensor of size 1]

epoch 0: iteration 70, the loss is Variable containing:
 0.1650
[torch.FloatTensor of size 1]

epoch 0: iteration 80, the loss is Variable containing:
 0.1600
[torch.FloatTensor of size 1]

epoch 1: iteration 0, the loss is Variable containing:
 0.1538
[torch.FloatTensor of size 1]

epoch 1: iteration 10, the loss is Variable containi

epoch 9: iteration 60, the loss is Variable containing:
 0.1703
[torch.FloatTensor of size 1]

epoch 9: iteration 70, the loss is Variable containing:
 0.1652
[torch.FloatTensor of size 1]

epoch 9: iteration 80, the loss is Variable containing:
 0.1558
[torch.FloatTensor of size 1]

(446409,)
(148803,)
test set auc is 0.6344029175708916; Val set auc is 0.6293206671721198


In [11]:
print(out.data.numpy()[:,1].shape)
print(out2.data.numpy()[:,1].shape)

(446409,)
(446409,)


In [20]:
print('test set auc is {}; Val set auc is {}'.format(score_1, score_2))


test set auc is 0.6344029175708916; Val set auc is 0.6293206671721198


In [19]:
import data_util
import data_preprocess
import datetime

tic= datetime.datetime.now()
train_data = data_util.load_train_data()
test_data= data_util.load_test_data()

naive_pre = data_preprocess.naive_preprocess()
y_test_index = test_data['id']
test_data.drop(['id'], axis=1, inplace=True)
train_data = naive_pre.dtype_change(train_data)
test_data = naive_pre.dtype_change(test_data)
#train_prop is specified
#X_train, X_val, y_train, y_val = data_util.split_train(train_data,prop=0.75)
y_train = train_data['target']
train_data.drop(['target','id'], axis=1, inplace=True)
X_train = train_data

X_train = naive_pre.scale(X_train)
X_test = naive_pre.scale(test_data,test=True)

#result = y_test_index,

In [26]:
X_test_tensor = torch.from_numpy(X_test.values)

X_test_tensor= X_test_tensor.float()

#print(X_train_tensor)
X_test_Variable = Variable(X_test_tensor)

out=  net(X_test_Variable)
out = F.softmax(out)
print(out)
prob = out.data.numpy()[:,1]

result = np.hstack((y_test_index, prob))

Variable containing:
 0.9771  0.0229
 0.9714  0.0286
 0.9794  0.0206
       ⋮        
 0.9646  0.0354
 0.9802  0.0198
 0.9744  0.0256
[torch.FloatTensor of size 892816x2]



In [28]:
np.savetxt("inter.csv", result, delimiter=",",header= 'id, target')